In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Imports section
import sys
sys.path.append('/content/drive/MyDrive/StackGAN/Modules')

import generate_dataset as gd # Self made dataset pipelining module
import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import os
from matplotlib import pyplot as plt
import pickle
import cv2
import time

**Miscellaneous Functions**

In [4]:
def save_image(image, filename, img_path):
  # Saves image at specified file path
  if os.path.exists(img_path)==False:
    os.makedirs(img_path)
  image=image.numpy().astype('uint8')
  cv2.imwrite((img_path+filename), image)

**Kulback Liebler Loss for Generator**

In [5]:
# KL Divergence is used as a regularization and latent space control parameter
def KL_adversarial_loss(y_true, y_pred):
  mean=y_pred[:, :128]
  logsigma=y_pred[:, 128:]
  return K.mean(-logsigma+0.5*(-1+K.exp(2.0*logsigma)+K.square(mean)))

**Conditional Augmentation**

In [6]:
# Define the conditional augmentation function
def conditional_augmentation(x):
  mean=x[:, :128]
  log_sigma=x[:, 128:]
  stddev = tf.exp(log_sigma)
  epsilon = tf.random.normal(tf.shape(mean), dtype=tf.float32)
  c = mean + stddev * epsilon
  return c

# Build the Conditional Augmentation Network (CAN) model
def build_ca_network(input_shape=(1024,), latent_dim=256):
  input_layer = tf.keras.layers.Input(shape=input_shape)
  x = tf.keras.layers.Dense(latent_dim)(input_layer)
  x = tf.keras.layers.LeakyReLU(x)
  ca = tf.keras.layers.Lambda(conditional_augmentation)(x)

  # Compiling the model
  model = tf.keras.models.Model(inputs=input_layer, outputs=ca)
  model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')

  return model

# Build an embedding compressor to 128 for Char_CNN_RNN
def embedding_compressor():
  input=tf.keras.layers.Input(shape=(1024,))
  compressed=tf.keras.layers.Dense(units=128)(input)
  compressed=tf.keras.layers.LeakyReLU(alpha=0.01)(compressed)

  model=tf.keras.Model(inputs=[input], outputs=[compressed])
  return model

**Stage 1 Generator Model**

In [7]:
# Upsampling block for generator
def upSample(x,dim):
  x=tf.keras.layers.Conv2DTranspose(dim, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x=tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)(x)
  x=tf.nn.relu(x)

  return x

# Stage 1 Generator builder
def Stage1_Generator():
  # Conditional Augmentation of  embedding vector
  inp=tf.keras.Input(shape=(1024,))
  ca=tf.keras.layers.Dense(256)(inp)
  ca=tf.keras.layers.LeakyReLU(alpha=0.01)(ca)
  c=tf.keras.layers.Lambda(conditional_augmentation)(ca)

  # Latent space conditioning
  noise=tf.keras.Input(shape=(100,))
  gen_input=tf.keras.layers.Concatenate(axis = 1)([c,noise])
  x=tf.keras.layers.Dense(units = 128*8*4*4, kernel_initializer = tf.keras.initializers.HeUniform())(gen_input)
  x=tf.keras.layers.LeakyReLU(alpha=0.01)(x)
  x=tf.keras.layers.Reshape(target_shape = (4, 4, 128*8), input_shape = (128*8*4*4, ))(x)
  x=tf.keras.layers.BatchNormalization(axis = -1, momentum = 0.99)(x)

  # Upsampling and deconvoluting latent space
  x=upSample(x,512)
  x=upSample(x,256)
  x=upSample(x,128)
  x=upSample(x,64)
  x=tf.keras.layers.Conv2DTranspose(3,kernel_size=3,strides=1,padding="same",kernel_initializer=tf.keras.initializers.GlorotUniform())(x)
  x=tf.nn.tanh(x)
  # Model
  stack1_gen=tf.keras.Model(inputs=[inp,noise], outputs=[x])
  return stack1_gen

In [8]:
generator=Stage1_Generator()
generator.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 dense (Dense)                  (None, 256)          262400      ['input_1[0][0]']                
                                                                                                  
 leaky_re_lu (LeakyReLU)        (None, 256)          0           ['dense[0][0]']                  
                                                                                                  
 lambda (Lambda)                (None, 128)          0           ['leaky_re_lu[0][0]']            
                                                                                              

**Stage 1 Discriminator**

In [9]:
# Build Stage 1 Discriminator
def Stage1_Discriminator():
  # Input image
  input1 = tf.keras.Input(shape=(64, 64, 3))
  x = tf.keras.layers.Conv2D(64, kernel_size=(4,4), strides=2, padding='same', use_bias=False, kernel_initializer=tf.keras.initializers.HeUniform())(input1)
  x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
  # Downsampling and convoluting image
  x = downSample(x, 128)
  x = downSample(x, 256)
  x = downSample(x, 512)

  # Concatenating compressing text embedding to generated distribution
  input2 = tf.keras.Input(shape=(4,4,128,)) # Text embedding
  x = tf.keras.layers.concatenate([x, input2])
  x = tf.keras.layers.Conv2D(512, kernel_size=(1,1), padding='same', strides=1, use_bias=False,kernel_initializer='he_uniform')(x)
  x = tf.keras.layers.BatchNormalization()(x)
  x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

  # Logit generation
  x= tf.keras.layers.Flatten()(x)
  x = tf.keras.layers.Dense(1, kernel_initializer=tf.keras.initializers.GlorotUniform())(x) # 0 or 1
  x = tf.keras.layers.Activation('sigmoid')(x)
  # Model
  stage1_dis = tf.keras.Model(inputs=[input1, input2], outputs=[x])
  return stage1_dis

# Downsampling block for Discriminator
def downSample(x, kernels, kernel_size=(4,4), strides=2, activation=True):
  x = tf.keras.layers.Conv2D(kernels, kernel_size=kernel_size, padding='same', strides=strides, use_bias=False, kernel_initializer=tf.keras.initializers.HeUniform())(x)
  x = tf.keras.layers.BatchNormalization()(x)

  if activation:
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
  return x

In [10]:
discriminator=Stage1_Discriminator()
discriminator.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 64)   3072        ['input_3[0][0]']                
                                                                                                  
 leaky_re_lu_2 (LeakyReLU)      (None, 32, 32, 64)   0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 16, 16, 128)  131072      ['leaky_re_lu_2[0][0]']          
                                                                                            

**Stage 1 Adversarial Model**

In [11]:
# Build Stage 1 Adversarial model
def Stage1_Adversarial(generator_model, discriminator_model):
    input1 = tf.keras.Input(shape=(1024,)) # Text embedding
    input2 = tf.keras.Input(shape=(100,)) # Noise
    input3 = tf.keras.Input(shape=(4,4,128,)) # Compressed embedding

    img = generator_model([input1, input2]) # Text, noise

  # Discriminator not trainable during adversarial game
    discriminator_model.trainable = False

  # Model
    discrimOutput = discriminator_model([img, input3])
    adversarial_model = tf.keras.Model(inputs=[input1, input2, input3], outputs=[discrimOutput])
    return adversarial_model

In [12]:
adversarial=Stage1_Adversarial(generator, discriminator)
adversarial.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 1024)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 model (Functional)             (None, 64, 64, 3)    10291843    ['input_5[0][0]',                
                                                                  'input_6[0][0]']                
                                                                                                  
 input_7 (InputLayer)           [(None, 4, 4, 128)]  0           []                         

**Stage 1 Model**

In [13]:
from keras.utils.version_utils import disallow_legacy_graph
class Stage1_GAN:
  def __init__(self, train_ds, epochs=350, z_dim=100, batch_size=64, gen_lr=0.001, dis_lr=0.001):
    self.epochs=epochs
    self.z_dim=z_dim
    self.batch_size=batch_size
    self.gen_lr=gen_lr
    self.dis_lr=dis_lr
    self.img_size=64
    # Training dataset
    self.train_ds=train_ds

    # Optimizers to the models
    self.gen_optimizer=tf.keras.optimizers.Adam(beta_1=0.6, beta_2=0.999, learning_rate=self.gen_lr)
    self.dis_optimizer=tf.keras.optimizers.Adam(beta_1=0.6, beta_2=0.999, learning_rate=self.dis_lr)

    # Models
    self.generator=Stage1_Generator()
    self.generator.compile(optimizer=self.gen_optimizer, loss='mse')
    self.discriminator=Stage1_Discriminator()
    self.discriminator.loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
    self.discriminator.compile(optimizer=self.dis_optimizer, loss=self.discriminator.loss_function)

    # Embedding Compressor
    self.embed_compressor=embedding_compressor()
    self.embed_compressor.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')

    # Adversarial Model
    self.model=Stage1_Adversarial(self.generator, self.discriminator)
    self.model.compile(loss=['binary_crossentropy', KL_adversarial_loss], loss_weights=[1., 2.], optimizer=self.gen_optimizer)

    self.checkpoint1 = tf.train.Checkpoint(gen_optimizer=self.gen_optimizer, discriminator_optimizer=self.dis_optimizer, generator=self.generator,discriminator=self.discriminator)

  def train_GAN(self):
    self.gen_loss_log=[]
    self.dis_loss_log=[]
    start=time.time()
    for i in range(self.epochs):
      gen_epoch_loss=[]
      dis_epoch_loss=[]
      print("<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>")
      print("<><><><><><><><> Started Epoch:",(i+1),"<><><><><><><><><><><><><><><><><><><>")

      # Current batch from dataset
      real_img, real_embed=next(self.train_ds)
      if((i+1)%75==0):
        K.set_value(self.gen_optimizer.learning_rate, self.gen_optimizer.learning_rate/2)
        K.set_value(self.dis_optimizer.learning_rate, self.dis_optimizer.learning_rate/2)
      # Iterating through mini-batches
      num_iters=125
      for iter in range(num_iters):
        if((iter+1)%5==0):
          print(":3 ", end='')

        # To train discriminator on real images-real captions
        if(iter!=0):
          real_img, real_embed=next(self.train_ds)
        real_labels= tf.random.uniform(shape=(self.batch_size, 1), minval = .9, maxval = 1.)

        # To train discriminator on real images-mismatched real captions
        mismatched_img=tf.roll(real_img, shift=1, axis=0)
        mismatched_labels= tf.random.uniform(shape=(self.batch_size, 1), minval = .9, maxval = 1.)

        # To train discriminator on fake images-real captions
        fake_img=self.generator.predict_on_batch([real_embed, np.random.normal(0, 1, size=(self.batch_size, self.z_dim))])
        fake_labels= tf.random.uniform(shape=(self.batch_size, 1), minval = 0., maxval = .1)

        # Real captions compressed and reshaped for discriminator
        compressed_embed=self.embed_compressor.predict_on_batch(real_embed)
        compressed_embed=tf.reshape(compressed_embed, (-1, 1, 1, 128))
        compressed_embed=tf.tile(compressed_embed, (1, 4, 4, 1))

        # Discriminator logits
        real_logits=self.discriminator.predict_on_batch([real_img, compressed_embed])
        fake_logits=self.discriminator.predict_on_batch([fake_img, compressed_embed])
        mismatched_logits=self.discriminator.predict_on_batch([mismatched_img, compressed_embed])

        # Loss for each logit
        loss_real=tf.reduce_mean(self.discriminator.loss_function(real_labels, real_logits))
        loss_fake=tf.reduce_mean(self.discriminator.loss_function(fake_labels, fake_logits))
        loss_mismatched=tf.reduce_mean(self.discriminator.loss_function(mismatched_labels, mismatched_logits))

        # Train Discriminator manually
        with tf.GradientTape() as dis_tape:
          # Total discriminator weighted loss
          dis_loss=0.5*tf.add(loss_real, 0.5*tf.add(loss_fake, loss_mismatched))

        # Discriminator gradient descent on optimizer
        discriminator_gradient=dis_tape.gradient(dis_loss, self.discriminator.trainable_variables)
        self.dis_optimizer.apply_gradients(zip(discriminator_gradient, self.discriminator.trainable_variables))
        dis_epoch_loss.append(["Batch:",(iter+1),"|| Loss:", dis_loss])

        # Training Adversarial GAN model
        gen_loss=self.model.train_on_batch([real_embed, np.random.normal(0, 1, size=(self.batch_size, self.z_dim)), compressed_embed], [real_labels])
        gen_epoch_loss.append(["Batch:",(iter+1),"|| Loss:", gen_loss])

      print("Discriminator's's Loss after epoch number",i,"is=",dis_loss)
      print("Generator's Loss after epoch number",i,"is=",gen_loss)

      # Saving 20 generator images after every 50 epochs
      if(i%50==0):
        gen_images=self.generator.predict_on_batch([real_embed, np.random.normal(0, 1, size=(self.batch_size, self.z_dim))])
        for num, image in enumerate(gen_images[:10]):
          image=127.5*image+127.5
          image=image.numpy().astype('uint8')
          save_image(image, f'{i+1}_{num}.jpg', '/content/drive/MyDrive/StackGAN/Stage_1/Generated_Images/')

      # Saving model weights after every 50 epochs
      if(i%50==0):
        self.generator.save_weights(f'/content/drive/MyDrive/StackGAN/Stage_1/Model_Weights/Gen_Epochs:{i+1}.h5')
        self.discriminator.save_weights(f'/content/drive/MyDrive/StackGAN/Stage_1/Model_Weights/Dis_Epochs:{i+1}.h5')

      # Appending losses
      self.gen_loss_log.append(gen_epoch_loss)
      self.dis_loss_log.append(dis_epoch_loss)

      print("Time elapsed:",(time.time()))
      print("<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>")

    with open('/content/drive/MyDrive/StackGAN/Stage_1/Model_Weights/Loss.txt', 'w') as file:
      # Writing Generator Loss Log file
      file.write('Generator Loss:\n')
      for item in self.gen_loss:
        item=str(item)+'\n'
        file.write(item)

      # Writing Discriminator Loss Log file
      file.write('Discriminator Loss:\n')
      for item in self.dis_loss:
        item=str(item)+'\n'
        file.write(item)

    self.generator.save_weights('/content/drive/MyDrive/StackGAN/Stage_1/Model_Weights/Gen_Epochs:All.h5')
    self.discriminator.save_weights('/content/drive/MyDrive/StackGAN/Stage_1/Model_Weights/Gen_Epochs:All.h5')

**Training Stage 1 GAN**

In [14]:
start_time=time.time()

# Getting the train dataset
dataset=gd.Dataset_Generator(img_size=(64, 64))
train=dataset.get_train_dataset()

# Iterator is passed to the model for training
train_iterator=iter(train)

In [ ]:
# Stage 1 GAN model
Stage_1_GAN=Stage1_GAN(train_iterator)
# Training the model
Stage_1_GAN.train_GAN()

print("Total time elapsed in training:", (time.time()-start_time))

<><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><><>
<><><><><><><><> Started Epoch: 1 <><><><><><><><><><><><><><><><><><><>
:3 

In [ ]:
drive.flush_and_unmount()